---
# Notebook for playing around with visualisation and segmentation of point clouds

Wouter Van den Broeck

---

## Import Libraries

In [1]:
# Import libraries
import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
import os
import pandas as pd
import random
from tqdm import tqdm

# Own files/libraries
from utils import read_clouds, combine_pcds, get_bbox

# Specify data directory
DATA_DIR = "/mnt/c/Users/wavdnbro/OneDrive - UGent/Documents/spacetwin/datasets/wythamwoods/"



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Get bounding boxes of tiles and segmented trees to find overlap

Specify paths

In [23]:
# Path to datafolders
datafolder_segm = bbox_path = os.path.join(DATA_DIR, 'segmented/')
datafolder_raw = bbox_path = os.path.join(DATA_DIR, 'raw/')

# Path to overall bounding box of segmented trees
bbox_extent = os.path.join(DATA_DIR, 'segmented/extent.csv')

# Path to bounding boxes of segmented trees
bbox_trees = "/mnt/c/Users/wavdnbro/OneDrive - UGent/Documents/spacetwin/datasets/wythamwoods/segmented/bounding_boxes.csv"

# Path to valid tiles within extent of segmented trees
valid_tiles_path = os.path.join(DATA_DIR, 'raw/valid_tiles.csv')

Find overall bounding box of all segmented trees

In [ ]:
# Read all segmented trees
pcds = read_clouds(datafolder_segm)

# Compute overall bounding box
get_bbox(pcds, compute_overall_bbox=True, path_out=bbox_extent)

# Compute single bounding boxes
get_bbox(pcds, path_out=bbox_trees)

Find valid tiles falling within bounding box of segmented trees

In [ ]:
# allowed file formats
FORMATS = ['ply']
    
# Get filenames
filenames = [f for f in os.listdir(datafolder_raw) if f[-3:] in FORMATS]

# Get bounding box of all segmented trees
bbox = pd.read_csv(bbox_extent)

# Read each tile and check if it falls within the boundary
filenames_ok = []
for i in tqdm(range(len(filenames))):
    filename = filenames[i]
    pcl = o3d.io.read_point_cloud(os.path.join(datafolder_raw, filename))
    bbox_tile = get_bbox(pcl)

    bbox_s = bbox.values[0]
    bbox_t = bbox_tile.values[0]

    if not ((bbox_t[3] < bbox_s[0]) | (bbox_t[0] > bbox_s[3]) | (bbox_t[4] < bbox_s[1]) | (bbox_t[1] > bbox_s[4])):
        filenames_ok.append(filename)

# Write names of valid tiles to csv file
columns = ['filename'] 
df_valid_tiles = pd.DataFrame(data=filenames_ok, columns=columns)
df_valid_tiles.to_csv(valid_tiles_path, index=False)


Clip valid tiles to within extent of trees

In [3]:

bbox = pd.read_csv(bbox_extent)
bb_x_min = bbox['x_min'].values[0]
bb_x_max = bbox['x_max'].values[0]
bb_y_min = bbox['y_min'].values[0]
bb_y_max = bbox['y_max'].values[0]

# Get tilenames
# datafolder_raw = "/mnt/c/Users/wavdnbro/OneDrive - UGent/Documents/spacetwin/datasets/wythamwoods/raw/"
# with open(datafolder_raw + "valid_tiles.txt", 'r') as f:
#     tilenames = f.readlines()

tilenames = pd.read_csv(valid_tiles_path)['filename'].values

# Define margine to clip tiles at boarder
m = 10
m_ymax = 25

# Datafolder to store clipped tiles
datafolder_raw_selection_clipped = "/mnt/c/Users/wavdnbro/OneDrive - UGent/Documents/spacetwin/datasets/wythamwoods/raw_selection_clipped_" + str(m) + '-' + str(m_ymax) + "/"
if not os.path.isdir(datafolder_raw_selection_clipped):
    os.mkdir(datafolder_raw_selection_clipped)

# Loop over tiles and clip to bounding box + margin
for i in tqdm(range(len(tilenames))):
    # Read tile
    tilename = tilenames[i]
    # tilename = tilename.split('\n')[0]
    tile = o3d.io.read_point_cloud(datafolder_raw + tilename)
    
    # Convert points to numpy array 
    points = np.asarray(tile.points)
    x = points[:,0]
    y = points[:,1]

    # Select only points within bounding box + margin
    points = points[((x > (bb_x_min + m)) & (x < (bb_x_max - m)) & (y > (bb_y_min + m)) & (y < (bb_y_max - m_ymax))), :]

    # Save tile (if non empty)
    if points.shape[0] > 0:
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(points)
        o3d.io.write_point_cloud(datafolder_raw_selection_clipped + tilename, pcd)


100%|██████████| 63/63 [16:56<00:00, 16.14s/it]


Combine multiple point clouds to one point cloud

In [11]:
# ---------------
# Save segmented trees and tiles as one file
# ---------------

# Combine tiles to one big point cloud and save to disk
# datafolder_raw_clipped = "/mnt/c/Users/wavdnbro/OneDrive - UGent/Documents/spacetwin/datasets/wythamwoods/raw_selection_clipped_10-25/"
# path_out = "/mnt/c/Users/wavdnbro/OneDrive - UGent/Documents/spacetwin/datasets/wythamwoods/raw_selection_clipped_10-25_single/wytham_winter_singlepcl.pcd"

# combine_pcds(datafolder_raw_clipped, path_out)

Reading point clouds...


100%|██████████| 48/48 [05:49<00:00,  7.28s/it]


combining point clouds...


100%|██████████| 48/48 [00:03<00:00, 12.42it/s]


Writing point cloud...


Attribute label to point cloud

In [7]:
def view1D(a, b): # a, b are arrays
    a = np.ascontiguousarray(a)
    b = np.ascontiguousarray(b)
    void_dt = np.dtype((np.void, a.dtype.itemsize * a.shape[1]))
    return a.view(void_dt).ravel(),  b.view(void_dt).ravel()


def isin_nd(a,b):
    # a,b are the 3D input arrays to give us "isin-like" functionality across them
    A,B = view1D(a.reshape(a.shape[0],-1),b.reshape(b.shape[0],-1))
    return np.isin(A,B)


# Get list of tiles
datafolder_raw_selection_clipped = "/mnt/c/Users/wavdnbro/OneDrive - UGent/Documents/spacetwin/datasets/wythamwoods/raw_selection_clipped_10-25/"
tilenames = [datafolder_raw_selection_clipped + f for f in os.listdir(datafolder_raw_selection_clipped) if f[-3:] == 'ply']
# tiles = read_clouds(tilenames)

# Get bounding boxes of segmented trees
bbox = pd.read_csv(bbox_trees)

# Make mapping of tree file to unique number 
filenames = [f for f in os.listdir(datafolder_segm) if f[-3:] == 'ply']
tree_file2number = {filename: i for i, filename in enumerate(filenames)}
tree_number2file = {i: filename for i, filename in enumerate(filenames)}

# Iterate over all tiles
for i in tqdm(range(len(tilenames))):

    # Read tile
    tilename = tilenames[i]
    tile = o3d.io.read_point_cloud(tilename)

    # Get bounds of tile
    points = np.asarray(tile.points)
    t_x_max = np.max(points[:,0])
    t_x_min = np.min(points[:,0])
    t_y_max = np.max(points[:,1])
    t_y_min = np.min(points[:,1])

    # Get trees that fall (partly) within bounds of the tile
    bbox_in = bbox[((bbox['x_min'] < t_x_max) & (bbox['x_max'] > t_x_min) & (bbox['y_min'] < t_y_max) & (bbox['y_max'] > t_y_min))]
    trees_names_in = [tree_number2file[i] for i in bbox_in.index]
    
    # Read in point clouds of included trees
    trees_in = read_clouds([datafolder_segm + tree_name for tree_name in trees_names_in])

    # Pre-allocate label array with value '-1'
    label = np.zeros((len(points), 2), dtype=np.int32) - 1

    # Iterate over all trees within tile
    for tree_name, tree_in in zip(trees_names_in, trees_in):
        # Boolean list indicating where tile points occur as tree points
        row_match = isin_nd(points, np.asarray(tree_in.points))

        # Allocate class and instance label
        label[row_match, 0] = 1
        label[row_match, 1] = tree_file2number[tree_name]

    # Save label as csv file
    columns = ['label_class', 'label_instance'] 
    label_df = pd.DataFrame(data=label, columns=columns)
    label_df.to_csv(tilename[:-4] + '_label.csv')
    


100%|██████████| 48/48 [1:23:25<00:00, 104.29s/it]


Colorize tile according to tree instances

In [20]:

# Compute number of trees
datafolder_segm = "/mnt/c/Users/wavdnbro/OneDrive - UGent/Documents/spacetwin/datasets/wythamwoods/segmented/"
nr_trees = len([f for f in os.listdir(datafolder_segm) if f[-3:] == 'ply'])

# Make colormap, mapping tree number to a unique color, -1 = non-tree
colors = np.random.randint(0, 256, (nr_trees, 3)) / 255
colormap = {i: color for i, color in enumerate(colors)}
colormap[-1] = np.random.randint(0, 256, 3) / 255

# Read in all tiles
datafolder_raw_clipped = "/mnt/c/Users/wavdnbro/OneDrive - UGent/Documents/spacetwin/datasets/wythamwoods/raw_selection_clipped_10-25/"
tilenames = [f for f in os.listdir(datafolder_raw_clipped) if f[-3:] == 'ply']
# tiles = [o3d.io.read_point_cloud(datafolder_raw_clipped + tile) for tile in tilenames]

# Loop over all tiles
tiles = []
for tilename in tilenames[:5]:
    # Read tile
    tile = o3d.io.read_point_cloud(datafolder_raw_clipped + tilename)

    # Read in file with labels for all points in tile
    labels = pd.read_csv(datafolder_raw_clipped + tilename[:-4] + '_label.csv')

    # Map point labels to color
    colors = [colormap[label] for label in labels['label_instance']]
    tile.colors = o3d.utility.Vector3dVector(colors)

    # Optional downsampling
    tile = tile.voxel_down_sample(0.1)    

    tiles.append(tile)

# o3d.visualization.draw_geometries(tiles)
tiles

[PointCloud with 350670 points.,
 PointCloud with 691314 points.,
 PointCloud with 742520 points.,
 PointCloud with 777477 points.,
 PointCloud with 814386 points.]

## Play around with normals and meshes

In [ ]:
datafolder = "/mnt/c/Users/wavdnbro/OneDrive - UGent/Documents/spacetwin/datasets/wythamwoods/segmented/"
filenames = os.listdir(datafolder) 

pcl = o3d.t.io.read_point_cloud(datafolder + filenames[0])
pcl.point.labels = o3d.core.Tensor(np.repeat(1, len(trees[0].points)), o3d.core.int32)
pcl.point.positions

In [ ]:
trees[0].geometry.estimate_normals(trees[0])

from open3d.cuda.pybind.geometry import 
from open3d.cuda.pybind.io import 



In [ ]:
trees[0].estimate_normals()

In [ ]:
np.asarray(trees[0].normals)

In [ ]:
distances = trees[0].compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 3 * avg_dist

print(avg_dist, radius)


In [ ]:
bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    trees[0],
    o3d.utility.DoubleVector([radius, radius * 2])
)
bpa_mesh

## Code for adding scanpositions to ply files

In [22]:
# -----------------
# Add scanposition (as normals) and time attribute to point cloud and save as ply file 
# using open3d
# -----------------

device = o3d.core.Device("CPU:0")
dtype = o3d.core.float32

pcd = o3d.t.geometry.PointCloud(device)

# Array with x, y, z positions
positions = [[0, 0, 0], [1, 1, 1], [2, 2, 2]]
pcd.point.positions = o3d.core.Tensor(
    positions,
    dtype,
    device,
)

n_points = len(positions)

# Single scan position coordinates
scanpos = [[5, 5, 5]]
scanpos = np.repeat(scanpos, n_points, axis=0)
pcd.point.normals = o3d.core.Tensor(
    scanpos,
    dtype,
    device
)

# Random time
time = [[0]]
time = np.repeat(time, n_points, axis=0)
pcd.point.time = o3d.core.Tensor(
    time,
    dtype,
    device
)

filename = 'tests/pcl_with_pos_as_normal.ply'
o3d.t.io.write_point_cloud(
    filename, 
    pcd, 
    write_ascii=False, 
    compressed=False, 
    print_progress=False,
)



True

## Try out reading rxp files

In [1]:
import pdal

DATA_DIR = "/mnt/c/Users/wavdnbro/OneDrive - UGent/Documents/spacetwin/datasets/eucface/"


json = """
{
    "pipeline": [
        "/mnt/c/Users/wavdnbro/OneDrive - UGent/Documents/spacetwin/datasets/eucface/220505_130358.rxp"
    ]
}
"""

pipeline = pdal.Pipeline(json)
count = pipeline.execute()
arrays = pipeline.arrays

arrays


[array([(4.27240038, -0.06492999,  2.48329592, 11057.87546044, 1, 1, 28.3     , -5.7574115, 5.04542504, 3., nan, 1, 0, 0.86435125, -1.27520496e-02,  0.50272685, -0.08861916, -0.00059048, -0.05317464, 42035),
        (4.27344704, -0.0649789 ,  2.48000431, 11057.87546209, 1, 1, 27.63    , -6.4271717, 5.04467431, 5., nan, 1, 0, 0.86469426, -1.27638835e-02,  0.50213635, -0.08865409, -0.00058926, -0.05310998, 40572),
        (4.27222824, -0.06499664,  2.47512722, 11057.87546384, 1, 1, 27.93    , -6.126057 , 5.04117089, 4., nan, 1, 0, 0.86506089, -1.27765324e-02,  0.50150414, -0.08869141, -0.00058796, -0.05304075, 41230),
        ...,
        (1.84184647, -0.00234389, -1.5467087 , 11148.07597837, 1, 1, 25.07    , -7.8121033, 2.49900012, 6., nan, 1, 0, 0.76452867, -5.33666122e-05, -0.64458948, -0.06871075, -0.00221053,  0.06412051, 37547),
        (1.84607422, -0.00231917, -1.55256999, 11148.07598012, 1, 1, 24.949999, -7.937126 , 2.50600012, 8., nan, 1, 0, 0.76405736, -4.27746745e-05, -0.6451

: 

In [1]:
RiVLib_DIR = "/home/woutervdb/spacetwin/pcl-segmentation/rivlib-2_6_0-arm_64-linux-gcc5:"

from dotenv import load_dotenv, dotenv_values
load_dotenv()

import os
os.environ['PDAL_DRIVER_PATH'] = RiVLib_DIR + os.environ['PDAL_DRIVER_PATH'] 
os.environ['PDAL_DRIVER_PATH']



'/home/woutervdb/spacetwin/pcl-segmentation/rivlib-2_6_0-arm_64-linux-gcc5:/home/woutervdb/anaconda3/envs/spacetwin/lib:/home/woutervdb/anaconda3/envs/spacetwin/lib/python3.10/site-packages/lib:/home/woutervdb/anaconda3/envs/spacetwin/lib/python3.10/site-packages/lib64:/home/woutervdb/anaconda3/envs/spacetwin/lib/python3.10/site-packages/pdal'